In [0]:
#!unzip robohearts.zip
%cd robohearts/

In [0]:
import gym
from gymhearts.Hearts import *
from mc import MC
from gymhearts.Agent.human import Human
from gymhearts.Agent.randomAI import RandomAI

In [0]:
NUM_ITERS = 10
NUM_EPISODES = 1000
TRAINING_ITERS = 10000
MAX_SCORE = 100
playersNameList = ['MC', 'Rando', 'Megumin', 'Darkness']
agent_list = [0, 0, 0, 0]

# Human vs Random
agent_list[0] = MC(playersNameList[0])
agent_list[1] = RandomAI(playersNameList[1])
agent_list[2] = RandomAI(playersNameList[2])
agent_list[3] = RandomAI(playersNameList[3])

In [0]:
%%capture 
# who cares about output in training
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
weights = []
for trn_episode in range(TRAINING_ITERS):
    observation = env.reset()
    history = []
    while True:
        now_event = observation['event_name']
        IsBroadcast = observation['broadcast']
        action = None
        if IsBroadcast == True:
            for agent in agent_list:
                agent.Do_Action(observation)
        else:
            playName = observation['data']['playerName']
            for agent in agent_list:
                if agent.name == playName:
                    action = agent.Do_Action(observation)

        # update my agent
        if now_event == 'RoundEnd':
            agent_list[0].update_reward_fn(history, -reward['MC'])
            history = []
            weights = agent_list[0].weight_vec
        if now_event == 'GameOver':
              break     
        observation2, reward, done, info = env.step(action)
        if not IsBroadcast and observation['data']['playerName'] == 'MC':
            history.append(observation)
        observation = observation2


In [0]:
# Evaluate the MC! Comment this line in for pretrained
weights = [ 0.03294658,  0.97709491, -0.05257657, -1.59303163,  0.86027217,  2.21585216,
 -1.70721793, -0.54153034, -0.58670556, -0.0079195,2 -2.25642152, -2.5887047,
 -0.97398101,  0.19535891,  1.03471521,  0.30540229,  0.79773432, -3.36570225,
  0.79243335, -1.91547041,  0.48521647, -0.03905389,  1.08328444, -1.4381379,
  0.35961876,  0.21143888,  0.9117904,  1.04141316, -1.28770156, -1.13500158,
  0.87145646, -0.11043938, -1.32192829, -0.93339591,  0.75019153, -1.54272215,
 -0.2644076,  -2.21429632,  0.82837715,  0.62784818, 0.61909315, -3.58542627,
 -4.71934151,  0.49217059, -2.00754422, -0.8649239,  -0.49443393, -1.49407945,
  0.91736213,  0.85249011, -1.60881422, -0.55375437]
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
agent_list[0] = MC(playersNameList[0], params={'weights' : weights})
mc_wins = []
for i in range(NUM_ITERS):
  num_won = 0
  for i_episode in range(NUM_EPISODES):
      observation = env.reset()
      while True:
          now_event = observation['event_name']
          IsBroadcast = observation['broadcast']
          action = None
          if IsBroadcast == True:
              for agent in agent_list:
                  agent.Do_Action(observation)
          else:
              playName = observation['data']['playerName']
              for agent in agent_list:
                  if agent.name == playName:
                      action = agent.Do_Action(observation)
          if now_event == 'GameOver':
              num_won += int(observation['data']['Winner'] == 'MC')
              break
          observation, reward, done, info = env.step(action)

  print("Finished iter: " + str(i))
  mc_wins.append(num_won)

In [0]:
%%capture
# Evaluate the Random!
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
agent_list[0] = RandomAI(playersNameList[0])
rand_wins = []
for i in range(NUM_ITERS):
  num_won = 0
  for i_episode in range(NUM_EPISODES):
      observation = env.reset()
      while True:
          now_event = observation['event_name']
          IsBroadcast = observation['broadcast']
          action = None
          if IsBroadcast == True:
              for agent in agent_list:
                  agent.Do_Action(observation)
          else:
              playName = observation['data']['playerName']
              for agent in agent_list:
                  if agent.name == playName:
                      action = agent.Do_Action(observation)
          if now_event == 'GameOver':
              num_won += int(observation['data']['Winner'] == 'MC')
              break
          observation, reward, done, info = env.step(action)
  rand_wins.append(num_won)

In [0]:
print("The weights are: " + str(weights))
print("MC won: " + str(mc_wins) + " times")
print("Rand won: " + str(rand_wins) + " times")